In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import nibabel as nb
import hrf_tools
import hcp_utils as hcp
from analysis import plot_results

import matplotlib.pyplot as plt

In [32]:
import numpy as np
import nibabel as nb
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from nilearn_plotting_custom import plot_surf
from PIL import Image
from PIL import ImageDraw
import npp
import hcp_utils as hcp
from hcp_tools import load_flatmaps_59k
from hcp_tools import load_meshes
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
sns.set("paper", "white")
#%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
def plot_results(scores,score_type,data_type,vertex_info,subject,feature,dataset,title):
    '''Inputs:
        scores = data to plot
        score_type = r2, r, p, z, d, raw
        data_type = 32k (3T) or 59k (7T)
        vertex info = None or the vertex info if it is 59k data beacuse hcp_utils doesnt by default
        subject = eg 100610 subject id for file naming
        feature = eg as_scores plotted feature for file naming
        dataset = eg merlin or HCP_7T which dataset?
        title = 
    '''
    scratch_dir = '../tmp'
#     scratch_dir = '/scratch/scratch/Fri/jsmentch/tmp'
#     if not os.path.exists(scratch_dir):
#         os.mkdir(scratch_dir)
    if score_type == 'r2':
        v=[0,0.5]
        threshold=0.01
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'r':
        v=[0,1]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'p':
        v=[0,0.05]
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'z':
        v=[-10,10]
        threshold=3
        symmetric_cmap=True
        cmap='cold_hot'
    if score_type == 'd':
        v=[0,10]
        threshold=3
        symmetric_cmap=True
        cmap='inferno'
    if score_type == 'raw':
        v=[-10,10]
        threshold=1
        symmetric_cmap=True
        cmap='cold_hot'
    save_dir=f'../outputs/figures/{dataset}/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)    
    if data_type == '59k':
        flatmeshes=load_flatmaps_59k() #load flatmaps
        surf_path_msm = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.L.inflated_1.6mm_MSMAll.59k_fs_LR.surf.gii'
        mesh59k_msm = load_meshes(example_filename=surf_path_msm) #load other meshes
        # get data from results in plotting format
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0, vertex_info=vertex_info)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0, vertex_info=vertex_info)
        # sulcal depth paths
        sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
        sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
        # params for view to plot
        params = [('flat_L',score_cortex_dataL,flatmeshes.flat_left,sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,flatmeshes.flat_right,sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,mesh59k_msm.very_inflated_left,sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,mesh59k_msm.very_inflated_right,sulc_right,'right'),\
        ]
    elif data_type == '32k':
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0)
    #     # sulcal depth paths
    #     sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
    #     sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
    #     # params for view to plot
        params = [('flat_L',score_cortex_dataL,hcp.mesh.flat_left,hcp.mesh.sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,hcp.mesh.flat_right,hcp.mesh.sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,hcp.mesh.very_inflated_left,hcp.mesh.sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,hcp.mesh.very_inflated_right,hcp.mesh.sulc_right,'right'),\
        ]
    # plot each hemi and mesh, save to outputs dir
    for name, data, mesh, sulc, hemi in params:
        #figure, axes = plt.subplots(subplot_kw=dict(projection="3d"), figsize=(6,4))
        plot_surf(mesh,\
                data, \
                  cmap=cmap,symmetric_cmap=symmetric_cmap, avg_method='median',#figure=fig,\
                bg_map=sulc, colorbar=True, vmin=v[0], vmax=v[1], threshold=threshold, hemi=hemi, \
                data_alpha=np.where(data>0,1,0),\
                data_remove=np.zeros(data.shape),output_file=f'{scratch_dir}/{name}.png')
#combine saved maps into one with PIL
#     if notebook==True:
    area = (75, 140, 635, 560) #area to crop from each image
#     else:
#         area = (105, 190, 880, 780)
        
    img = Image.open(f'{scratch_dir}/flat_L.png',mode='r')
    img = img.resize((770,720))
    cropped = img.crop(area)
    fL=cropped.transpose(Image.FLIP_LEFT_RIGHT)
    w,h = img.size
    c_area = (690, 0, w-10, h) # area of colorbar to crop
    cbar = img.crop(c_area)

    img = Image.open(f'{scratch_dir}/flat_R.png',mode='r')
    img = img.resize((770,720))
    fR = img.crop(area)

    img = Image.open(f'{scratch_dir}/vinf_L.png',mode='r')
    img = img.resize((770,720))
    iL = img.crop(area)
    #iL=cropped.transpose(Image.FLIP_LEFT_RIGHT)

    img = Image.open(f'{scratch_dir}/vinf_R.png',mode='r')
    img = img.resize((770,720))
    iR = img.crop(area)

    w,h=iR.size

    new_im = Image.new('RGB', ( (w*2)+70 , h*2) ,(255, 255, 255, 1))
    new_im.paste(fL,(0,h))
    new_im.paste(fR,(w,h))
    new_im.paste(iL,(0,0))
    new_im.paste(iR,(w,0))
    new_im.paste(cbar,(w*2,int(round(h/4))))

    w,h=new_im.size

    draw = ImageDraw.Draw(new_im)
    draw.text((0,0),f"{title}_{subject}_{feature}_{score_type}",(0,0,0))

    new_im.save(f'{save_dir}{title}_{subject}_{feature}_{score_type}.png')

In [2]:
#im_file = f'../sourcedata/data/budapest/brain/ds003017/derivatives/fmriprep/derivatives/cleaned/smoothed/sub-sid000005_run1_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii'
im_file = f'../../speech_face_analysis/data/fitlins_cifti/fitlins/task-MerlinMovie_space-fsLR_contrast-speech_stat-z_statmap.dscalar.nii'
#im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
img = nb.load(im_file)
img_y = img.get_fdata()
#img_y[np.isnan(img_y)] = 0
Y=img_y

vertex_info = hcp.get_HCP_vertex_info(img)


In [ ]:
plot_results(Y[0],'z','32k',vertex_info=vertex_info,subject='group',feature='speech', dataset='merlin',title=f'glm' )

In [40]:
#im_file = f'../sourcedata/data/budapest/brain/ds003017/derivatives/fmriprep/derivatives/cleaned/smoothed/sub-sid000005_run1_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii'
im_file = f'../../speech_face_analysis/data/fitlins_cifti/fitlins/task-MerlinMovie_space-fsLR_contrast-anyFaces_stat-z_statmap.dscalar.nii'
#im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
img = nb.load(im_file)
img_y = img.get_fdata()
#img_y[np.isnan(img_y)] = 0
Y=img_y

vertex_info = hcp.get_HCP_vertex_info(img)
plot_results(Y[0],'z','32k',vertex_info=vertex_info,subject='group',feature='any_faces', dataset='merlin',title=f'glm' )

/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[0.1743319  0.17924121 0.18096912 ... 0.81205445 0.81735    0.82800209]
[0.19327798 0.19757743 0.19782137 ... 0.79031321 0.79250868 0.79252901]
[0.1743319  0.17924121 0.18096912 ... 0.81205445 0.81735    0.82800209]
[0.19327798 0.19757743 0.19782137 ... 0.79031321 0.79250868 0.79252901]


In [41]:
#im_file = f'../sourcedata/data/budapest/brain/ds003017/derivatives/fmriprep/derivatives/cleaned/smoothed/sub-sid000005_run1_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii'
im_file = f'../../speech_face_analysis/data/fitlins_cifti/fitlins/task-MerlinMovie_space-fsLR_contrast-faceAndSpeech_stat-z_statmap.dscalar.nii'
#im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
img = nb.load(im_file)
img_y = img.get_fdata()
#img_y[np.isnan(img_y)] = 0
Y=img_y

vertex_info = hcp.get_HCP_vertex_info(img)
plot_results(Y[0],'z','32k',vertex_info=vertex_info,subject='group',feature='face_speech', dataset='merlin',title=f'glm' )

/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[0.14582987 0.15315018 0.16070432 ... 0.79143999 0.7933514  0.80179009]
[0.18549169 0.18847064 0.18904    ... 0.78097806 0.78253362 0.78632595]
[0.14582987 0.15315018 0.16070432 ... 0.79143999 0.7933514  0.80179009]
[0.18549169 0.18847064 0.18904    ... 0.78097806 0.78253362 0.78632595]


In [38]:
#im_file = f'../sourcedata/data/budapest/brain/ds003017/derivatives/fmriprep/derivatives/cleaned/smoothed/sub-sid000005_run1_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii'
im_file = f'../../speech_face_analysis/data/fitlins_cifti_nan/fitlins/task-MerlinMovie_space-fsLR_contrast-faceAndSpeech_stat-z_statmap.dscalar.nii'
#im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
img = nb.load(im_file)
img_y = img.get_fdata()
#img_y[np.isnan(img_y)] = 0
Y=img_y

vertex_info = hcp.get_HCP_vertex_info(img)
plot_results(Y[0],'z','32k',vertex_info=vertex_info,subject='group',feature='face_speech', dataset='merlin',title=f'glm_dec_z' )

[0.14584446 0.15315292 0.16070086 ...        nan        nan        nan]
[0.18548929 0.18846302 0.18903682 ...        nan        nan        nan]
[0.14584446 0.15315292 0.16070086 ...        nan        nan        nan]
[0.18548929 0.18846302 0.18903682 ...        nan        nan        nan]


## plot character encoding model results

In [7]:
Y=np.load('../outputs/char_encoding_train_all_test_all.npy')
plot_results(Y,'r2','32k',vertex_info=vertex_info,subject='group',feature='char', dataset='merlin',title=f'train_all_test_all_thresh' )

[0.00000000e+00 8.24201966e-07 2.26300302e-06 ... 5.32979877e-01
 5.39006600e-01 5.57462448e-01]
[0.00000000e+00 1.17110346e-06 1.68476191e-06 ... 5.07091557e-01
 5.12947641e-01 5.13103837e-01]
[0.00000000e+00 8.24201966e-07 2.26300302e-06 ... 5.32979877e-01
 5.39006600e-01 5.57462448e-01]
[0.00000000e+00 1.17110346e-06 1.68476191e-06 ... 5.07091557e-01
 5.12947641e-01 5.13103837e-01]


In [8]:
Y=np.load('../outputs/char_encoding_train_all.npy')
plot_results(Y,'r2','32k',vertex_info=vertex_info,subject='group',feature='char', dataset='merlin',title=f'train_all_test_one_thresh' )

[0.00000000e+00 9.56456634e-06 1.45001426e-05 ... 7.41044472e-01
 7.59792158e-01 7.75747210e-01]
[0.00000000e+00 3.76012762e-06 4.27633961e-06 ... 7.86080263e-01
 7.92836706e-01 8.12508197e-01]
[0.00000000e+00 9.56456634e-06 1.45001426e-05 ... 7.41044472e-01
 7.59792158e-01 7.75747210e-01]
[0.00000000e+00 3.76012762e-06 4.27633961e-06 ... 7.86080263e-01
 7.92836706e-01 8.12508197e-01]


In [9]:
Y=np.load('../outputs/char_encoding_train_one.npy')
plot_results(Y,'r2','32k',vertex_info=vertex_info,subject='group',feature='char', dataset='merlin',title=f'train_one_test_one_thresh' )

[0.00000000e+00 3.18216402e-05 7.09235171e-05 ... 9.63270054e-01
 9.80186634e-01 9.97936896e-01]
[0.00000000e+00 1.55418429e-05 2.52948558e-05 ... 9.88209520e-01
 9.88875569e-01 1.00079341e+00]
[0.00000000e+00 3.18216402e-05 7.09235171e-05 ... 9.63270054e-01
 9.80186634e-01 9.97936896e-01]
[0.00000000e+00 1.55418429e-05 2.52948558e-05 ... 9.88209520e-01
 9.88875569e-01 1.00079341e+00]


In [8]:
Y=np.load('../outputs/char_encoding_train_one2.npy')
plot_results(Y,'r2','32k',vertex_info=vertex_info,subject='group',feature='char', dataset='merlin',title=f'train_one_test_one2' )

/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[0.00000000e+00 3.67677658e-05 9.21799309e-05 ... 8.72929386e-01
 8.94747575e-01 9.11597611e-01]
[0.00000000e+00 7.64043521e-06 1.03407433e-05 ... 9.97475954e-01
 9.98270789e-01 1.00784540e+00]
[0.00000000e+00 3.67677658e-05 9.21799309e-05 ... 8.72929386e-01
 8.94747575e-01 9.11597611e-01]
[0.00000000e+00 7.64043521e-06 1.03407433e-05 ... 9.97475954e-01
 9.98270789e-01 1.00784540e+00]


## hcp plot group encoding

In [3]:
import nibabel as nb
import numpy as np

import nilearn.plotting as nlp
from nilearn_plotting_custom import plot_surf
import hcp_utils as hcp
from hcp_tools import extract_cortex
from hcp_tools import load_meshes
#from analysis import load_data
from hcp_tools import load_flatmaps_59k
from analysis import simple_ridgeCV
#from analysis import plot_59k_results
import os
import hrf_tools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set("paper", "white")
%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

In [5]:
subject=100610
feature='as_scores'
n_movies=1

def load_data_AS(subject,feature,n_movies):
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import MinMaxScaler
    # Inputs: subject = HCP id eg 100610
    #         feature='mfs'
    #         n_movies is a number 1-4
    # Returns: X feature data (2D; time x feature)
    #          Y brain data (2D; time x grayordinate)
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    y_l=[]
    x_l=[]
    stim = ['tfMRI_MOVIE1_7T_AP','tfMRI_MOVIE2_7T_PA','tfMRI_MOVIE3_7T_PA','tfMRI_MOVIE4_7T_AP']
    stim_feat = ['7T_MOVIE1_CC1_v2', '7T_MOVIE2_HO1_v2', '7T_MOVIE3_CC2_v2', '7T_MOVIE4_HO2_v2']
    
    for i in np.arange(n_movies):
        #load brain image
        im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/{str(subject)}/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
        img = nb.load(im_file)
        img_y = img.get_fdata()
        img_y = scaler.fit_transform(img_y)
        #img_y = scaler.transform(img_y)
        
        #load feature
        feat_x = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_{feature}.npy')
        feat_x = hrf_tools.apply_optimal_hrf_10hz(feat_x,2.08)
        #feat_x = hrf_tools.resample_1hz(feat_x)

        from scipy.signal import resample
        feat_x = resample(feat_x, img_y.shape[0], axis=0)

        #feat_x = feat_x[:img_y.shape[0],:]
        #feat_x=feat_x.T
        y_l.append(img_y)
        x_l.append(feat_x)
    Y=np.vstack(y_l)
    X=np.vstack(x_l)
    scaler = StandardScaler()

    #X = scaler.fit_transform(X)
    vertex_info = hcp.get_HCP_vertex_info(img)
    return X,Y,vertex_info

X,Y,vertex_info = load_data_AS(subject,feature,n_movies)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [8]:
import glob

feature='as_scores'
dataset='HCP_7T'

#clean_path = f'../outputs/encoding_model/{dataset}/{feature}/'
flist = glob.glob(f'../outputs/encoding_model/{dataset}/{feature}/*scores.npy')
#all_=[]
all_sub = []
for s in flist:
    cur = np.load(f'{s}')
    all_sub.append(cur)
    

all_sub = np.array(all_sub)

all_sub_mean = np.mean(all_sub,axis=0)


#vertex_info = hcp.get_HCP_vertex_info(img)
plot_results(all_sub_mean,'r2','59k',vertex_info,"all",feature,dataset,'ridgeCV')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data


/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[-0.15546215 -0.15077404 -0.14981797 ...  0.09955957  0.09981586
  0.09995978]
[-0.19606981 -0.19449887 -0.19166744 ...  0.08164116  0.08302976
  0.08354594]
[-0.15546215 -0.15077404 -0.14981797 ...  0.09955957  0.09981586
  0.09995978]
[-0.19606981 -0.19449887 -0.19166744 ...  0.08164116  0.08302976
  0.08354594]


In [34]:
subject=192641
#subject=105923

scores=np.load(f'../outputs/encoding_model/HCP_7T/as_scores/ridgeCV_{subject}_as_scores_scores.npy')
weights = np.load(f'../outputs/encoding_model/HCP_7T/as_scores/ridgeCV_{subject}_as_scores_weights.npy')
corr = np.load(f'../outputs/encoding_model/HCP_7T/as_scores/ridgeCV_{subject}_as_scores_corr.npy')

music_weight=weights[:,132]
speech_weight=weights[:,0]
animal_weight=weights[:,67]

In [14]:
#old!!
import numpy as np
import nibabel as nb
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from nilearn_plotting_custom import plot_surf
from PIL import Image
from PIL import ImageDraw
import npp
import hcp_utils as hcp
from hcp_tools import load_flatmaps_59k
from hcp_tools import load_meshes
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
sns.set("paper", "white")
#%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
def plot_results(scores,score_type,data_type,vertex_info,subject,feature,dataset,title):
    '''Inputs:
        scores = data to plot
        score_type = r2, r, p, z, d, raw
        data_type = 32k (3T) or 59k (7T)
        vertex info = None or the vertex info if it is 59k data beacuse hcp_utils doesnt by default
        subject = eg 100610 subject id for file naming
        feature = eg as_scores plotted feature for file naming
        dataset = eg merlin or HCP_7T which dataset?
        title = 
    '''
    scratch_dir = '../tmp'
#     scratch_dir = '/scratch/scratch/Fri/jsmentch/tmp'
#     if not os.path.exists(scratch_dir):
#         os.mkdir(scratch_dir)
    if score_type == 'r2':
        v=[0,0.5]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'r':
        v=[0,1]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'p':
        v=[0,0.05]
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'z':
        v=[-10,10]
        threshold=3
        symmetric_cmap=True
        cmap='cold_hot'
    if score_type == 'd':
        v=[0,10]
        threshold=3
        symmetric_cmap=True
        cmap='inferno'
    if score_type == 'raw':
        v=[-1.5,1.5]
        threshold=None
        symmetric_cmap=True
        cmap='cold_hot'
    save_dir=f'../outputs/figures/{dataset}/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)    
    if data_type == '59k':
        flatmeshes=load_flatmaps_59k() #load flatmaps
        surf_path_msm = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.L.inflated_1.6mm_MSMAll.59k_fs_LR.surf.gii'
        mesh59k_msm = load_meshes(example_filename=surf_path_msm) #load other meshes
        # get data from results in plotting format
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0, vertex_info=vertex_info)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0, vertex_info=vertex_info)
        # sulcal depth paths
        sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
        sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
        # params for view to plot
        params = [('flat_L',score_cortex_dataL,flatmeshes.flat_left,sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,flatmeshes.flat_right,sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,mesh59k_msm.very_inflated_left,sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,mesh59k_msm.very_inflated_right,sulc_right,'right'),\
        ]
    elif data_type == '32k':
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0)
    #     # sulcal depth paths
    #     sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
    #     sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
    #     # params for view to plot
        params = [('flat_L',score_cortex_dataL,hcp.mesh.flat_left,hcp.mesh.sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,hcp.mesh.flat_right,hcp.mesh.sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,hcp.mesh.very_inflated_left,hcp.mesh.sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,hcp.mesh.very_inflated_right,hcp.mesh.sulc_right,'right'),\
        ]
    # plot each hemi and mesh, save to outputs dir
    for name, data, mesh, sulc, hemi in params:
        #figure, axes = plt.subplots(subplot_kw=dict(projection="3d"), figsize=(6,4))
        plot_surf(mesh,\
                data, \
                  cmap=cmap,symmetric_cmap=symmetric_cmap, avg_method='median',#figure=fig,\
                bg_map=sulc, colorbar=True, vmin=v[0], vmax=v[1], threshold=threshold, hemi=hemi, \
                data_alpha=np.where(data>0,1,1),\
                data_remove=np.zeros(data.shape),output_file=f'{scratch_dir}/{name}.png')
#combine saved maps into one with PIL
#     if notebook==True:
    area = (75, 140, 635, 560) #area to crop from each image
#     else:
#         area = (105, 190, 880, 780)
        
    img = Image.open(f'{scratch_dir}/flat_L.png',mode='r')
    img = img.resize((770,720))
    cropped = img.crop(area)
    fL=cropped.transpose(Image.FLIP_LEFT_RIGHT)
    w,h = img.size
    c_area = (690, 0, w-10, h) # area of colorbar to crop
    cbar = img.crop(c_area)

    img = Image.open(f'{scratch_dir}/flat_R.png',mode='r')
    img = img.resize((770,720))
    fR = img.crop(area)

    img = Image.open(f'{scratch_dir}/vinf_L.png',mode='r')
    img = img.resize((770,720))
    iL = img.crop(area)
    #iL=cropped.transpose(Image.FLIP_LEFT_RIGHT)

    img = Image.open(f'{scratch_dir}/vinf_R.png',mode='r')
    img = img.resize((770,720))
    iR = img.crop(area)

    w,h=iR.size

    new_im = Image.new('RGB', ( (w*2)+70 , h*2) ,(255, 255, 255, 1))
    new_im.paste(fL,(0,h))
    new_im.paste(fR,(w,h))
    new_im.paste(iL,(0,0))
    new_im.paste(iR,(w,0))
    new_im.paste(cbar,(w*2,int(round(h/4))))

    w,h=new_im.size

    draw = ImageDraw.Draw(new_im)
    draw.text((0,0),f"{title}_{subject}_{feature}_{score_type}",(0,0,0))

    new_im.save(f'{save_dir}{title}_{subject}_{feature}_{score_type}.png')

In [15]:
plot_results(animal_weight,'raw','59k',vertex_info,subject,feature,dataset,'ridgeCV_animal_weight_thresh')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data
[-1.02260832 -0.98855968 -0.96570519 ...  1.95231854  1.95864161
  2.00993231]
[-0.94431338 -0.94313325 -0.9357573  ...  1.95445844  1.95812154
  1.96603391]
[-1.02260832 -0.98855968 -0.96570519 ...  1.95231854  1.95864161
  2.00993231]
[-0.94431338 -0.94313325 -0.9357573  ...  1.95445844  1.95812154
  1.96603391]


In [16]:
plot_results(music_weight,'raw','59k',vertex_info,subject,feature,dataset,'ridgeCV_music_weight')
plot_results(speech_weight,'raw','59k',vertex_info,subject,feature,dataset,'ridgeCV_speech_weight')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data
[0.02926834 0.03190657 0.0327309  ... 0.90649062 0.90804672 0.92063252]
[0.10509504 0.11103206 0.11376734 ... 0.78836263 0.7928823  0.80054868]
[0.02926834 0.03190657 0.0327309  ... 0.90649062 0.90804672 0.92063252]
[0.10509504 0.11103206 0.11376734 ... 0.78836263 0.7928823  0.80054868]
Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data
[0.24166978 0.24260728 0.24288854 ... 0.97544058 0.98519983 0.98621906]
[0.23217668 0.23998475 0.24000262 ... 0.95478453 0.9578788  0.9641425 ]
[0.24166978 0.24260728 0.24288854 ... 0.97544058 0.98519983 0.98621906]
[0.23217668 0.23998475 0.24000262 ... 0.95478453 0.9578788  0.9641425 ]


In [35]:
#plot_results(corr,'r','59k',vertex_info,subject,feature,dataset,'ridgeCV_as_corr')
plot_results(scores,'r2','59k',vertex_info,subject,feature,dataset,'ridgeCV_as_r2_scores')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data
[-0.46633811 -0.46141254 -0.4540893  ...  0.44455788  0.44665544
  0.4502424 ]
[-0.74588741 -0.72454169 -0.64685502 ...  0.44073376  0.44169705
  0.47836799]
[-0.46633811 -0.46141254 -0.4540893  ...  0.44455788  0.44665544
  0.4502424 ]
[-0.74588741 -0.72454169 -0.64685502 ...  0.44073376  0.44169705
  0.47836799]


In [29]:
scores.max()

0.215629261775001

In [36]:
#NEW!!
import numpy as np
import nibabel as nb
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from nilearn_plotting_custom import plot_surf
from PIL import Image
from PIL import ImageDraw
import npp
import hcp_utils as hcp
from hcp_tools import load_flatmaps_59k
from hcp_tools import load_meshes
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
sns.set("paper", "white")
#%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
def plot_results(scores,data_alpha,score_type,data_type,vertex_info,subject,feature,dataset,title):
    '''Inputs:
        scores = data to plot
        score_type = r2, r, p, z, d, raw
        data_type = 32k (3T) or 59k (7T)
        vertex info = None or the vertex info if it is 59k data beacuse hcp_utils doesnt by default
        subject = eg 100610 subject id for file naming
        feature = eg as_scores plotted feature for file naming
        dataset = eg merlin or HCP_7T which dataset?
        title = 
    '''
    scratch_dir = '../tmp'
#     scratch_dir = '/scratch/scratch/Fri/jsmentch/tmp'
#     if not os.path.exists(scratch_dir):
#         os.mkdir(scratch_dir)
    if score_type == 'r2':
        v=[0,0.5]
        threshold=0.001
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'r':
        v=[0,1]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'weights':
        v=[0,6]
        threshold=None
        symmetric_cmap=False
        cmap='tab10'
    if score_type == 'p':
        v=[0,0.05]
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'z':
        v=[-10,10]
        threshold=3
        symmetric_cmap=True
        cmap='cold_hot'
    if score_type == 'd':
        v=[0,10]
        threshold=3
        symmetric_cmap=True
        cmap='inferno'
    if score_type == 'raw':
        v=[-1.5,1.5]
        threshold=None
        symmetric_cmap=True
        cmap='cold_hot'
    save_dir=f'../outputs/figures/{dataset}/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)    
    if data_type == '59k':
        flatmeshes=load_flatmaps_59k() #load flatmaps
        surf_path_msm = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.L.inflated_1.6mm_MSMAll.59k_fs_LR.surf.gii'
        mesh59k_msm = load_meshes(example_filename=surf_path_msm) #load other meshes
        # get data from results in plotting format
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0, vertex_info=vertex_info)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0, vertex_info=vertex_info)
        
        alpha_cortex_dataL = hcp.left_cortex_data(data_alpha, fill=0, vertex_info=vertex_info)
        alpha_cortex_dataR = hcp.right_cortex_data(data_alpha, fill=0, vertex_info=vertex_info)
        
        # sulcal depth paths
        sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
        sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
        # params for view to plot
        params = [('flat_L',score_cortex_dataL,alpha_cortex_dataL,flatmeshes.flat_left,sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,alpha_cortex_dataR,flatmeshes.flat_right,sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,alpha_cortex_dataL,mesh59k_msm.very_inflated_left,sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,alpha_cortex_dataR,mesh59k_msm.very_inflated_right,sulc_right,'right'),\
        ]
    elif data_type == '32k':
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0)
    #     # sulcal depth paths
    #     sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
    #     sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
    #     # params for view to plot
        params = [('flat_L',score_cortex_dataL,alpha_cortex_dataL,hcp.mesh.flat_left,hcp.mesh.sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,hcp.mesh.flat_right,hcp.mesh.sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,alpha_cortex_dataR,alpha_cortex_dataL,hcp.mesh.very_inflated_left,hcp.mesh.sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,alpha_cortex_dataR,hcp.mesh.very_inflated_right,hcp.mesh.sulc_right,'right'),\
        ]
    # plot each hemi and mesh, save to outputs dir
    for name, data, alpha, mesh, sulc, hemi in params:
        alpha = np.where(alpha<0,0,alpha)
        alpha = np.where(alpha>1,1,alpha)*2
        #figure, axes = plt.subplots(subplot_kw=dict(projection="3d"), figsize=(6,4))
        plot_surf(mesh,\
                data, \
                  cmap=cmap,symmetric_cmap=symmetric_cmap, avg_method='median',#figure=fig,\
                bg_map=sulc, colorbar=True, vmin=v[0], vmax=v[1], threshold=threshold, hemi=hemi, \
#                data_alpha=np.where(data_alpha<0,0,data_alpha),\
                data_alpha=np.where(alpha<0,0,alpha),\
                data_remove=np.zeros(data.shape),output_file=f'{scratch_dir}/{name}.png')
#combine saved maps into one with PIL
#     if notebook==True:
    area = (75, 140, 635, 560) #area to crop from each image
#     else:
#         area = (105, 190, 880, 780)
        
    img = Image.open(f'{scratch_dir}/flat_L.png',mode='r')
    img = img.resize((770,720))
    cropped = img.crop(area)
    fL=cropped.transpose(Image.FLIP_LEFT_RIGHT)
    w,h = img.size
    c_area = (690, 0, w-10, h) # area of colorbar to crop
    cbar = img.crop(c_area)

    img = Image.open(f'{scratch_dir}/flat_R.png',mode='r')
    img = img.resize((770,720))
    fR = img.crop(area)

    img = Image.open(f'{scratch_dir}/vinf_L.png',mode='r')
    img = img.resize((770,720))
    iL = img.crop(area)
    #iL=cropped.transpose(Image.FLIP_LEFT_RIGHT)

    img = Image.open(f'{scratch_dir}/vinf_R.png',mode='r')
    img = img.resize((770,720))
    iR = img.crop(area)

    w,h=iR.size

    new_im = Image.new('RGB', ( (w*2)+70 , h*2) ,(255, 255, 255, 1))
    new_im.paste(fL,(0,h))
    new_im.paste(fR,(w,h))
    new_im.paste(iL,(0,0))
    new_im.paste(iR,(w,0))
    new_im.paste(cbar,(w*2,int(round(h/4))))

    w,h=new_im.size

    draw = ImageDraw.Draw(new_im)
    draw.text((0,0),f"{title}_{subject}_{feature}_{score_type}",(0,0,0))

    new_im.save(f'{save_dir}{title}_{subject}_{feature}_{score_type}.png')

In [12]:
#plot_results(music_weight,scores,'raw','59k',vertex_info,subject,feature,dataset,'ridgeCV_music_weight_alpha')
#plot_results(speech_weight,scores,'raw','59k',vertex_info,subject,feature,dataset,'ridgeCV_speech_weight_alpha')
plot_results(animal_weight,scores,'raw','59k',vertex_info,subject,feature,dataset,'ridgeCV_animal_weight_alpha')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data
[-0.73010646 -0.63736212 -0.62569156 ...  1.80473028  1.8320359
  1.86027779]
[-0.8171861  -0.69420576 -0.66384539 ...  1.88817272  1.9233567
  1.98140834]
[-0.73010646 -0.63736212 -0.62569156 ...  1.80473028  1.8320359
  1.86027779]
[-0.8171861  -0.69420576 -0.66384539 ...  1.88817272  1.9233567
  1.98140834]


In [37]:
weights.shape

(170494, 521)

In [38]:
most_likely=np.argmax(weights,axis=1)
most_likely.shape

(170494,)

In [39]:
as_classes = np.load('../../speech_face_analysis/notebooks/data/as_classes.npy')
for i,m in enumerate(most_likely):
    most_likely[i]=as_classes[m]

In [40]:
weights_category = []
for c in np.unique(as_classes):
    c_ind = np.where(as_classes==c)[0] # get indices of each high level as category
    #weights_category.append(np.mean(weights[:,c_ind], axis=1)) #append the MEAN weights of each category to the list
    weights_category.append(np.max(weights[:,c_ind], axis=1)) #append the MAX weights of each category to the list
weights_category=np.asarray(weights_category)
most_likely=np.argmax(weights_category,axis=0)

In [41]:
plot_results(most_likely,scores,'weights','59k',vertex_info,subject,'',dataset,'ridgeCV_weights_max')

Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]
